In [20]:
import json
from natasha import NamesExtractor
import re
import numpy as np
import pickle

In [21]:
path_df = '..\\02. Data creation\\df.pickle'

In [22]:
with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [23]:
df.head()

,topic,direction,text
0,Какой поступок вы назвали бы самым трусливым?,трусость характер страх слабость смелость храб...,С понятием трусости у меня идет прямая ассоциа...
1,Почему общество часто не ценит великих людей?,человек общество признание уважение успех,В истории нашей страны существует немало лично...
2,Какие цели важно ставить на жизненном пути?,цель жизнь смысл путь задача стремление мечта ...,"Как бы это банально не звучало, но мы живем дл..."
3,Что значит быть отзывчивым?,отзывчивость чувство доброта чуткость помощь д...,"Отзывчивость – это емкое понятие, вмещающее в ..."
4,Всегда ли хороша верность?,преданность любовь доверие дружба надёжность д...,"Верность, доверие, любовь, уважение, поддержка..."


### Создадим Bag of Words для именованных сущностей

In [24]:
# Функция расчета схожести строк по коэффициенту Жаккара
def dist_jaccard(string1, string2):
    
    str1 = string1
    str2= string2
    a = len(str1)
    b = len(str2)
    c = 0
    
    while str1 != '':
        
        s = str1[0]
        r = re.compile(s)
        c += min(len(r.findall(str1)), len(r.findall(str2)))
        str1 = r.sub('', str1)
        str2 = r.sub('', str2)
        
    return c / (a + b - c)

# Сравнение двух именованных сущностей
def compare_str(str1, str2):
    s1 = str1.split()
    s2 = str2.split()
    if len(s1) < len(s2):
        s1, s2 = s2, s1
    n = len(s1)
    m = len(s2)
    result = False
    b1 = re.search('[\"](.+)[\"]', str1)
    b2 = re.search('[\"](.+)[\"]', str2)
    if b1 is not None and b2 is not None:
        if b1.group(1) == b2.group(1):
            return True
        else:
            return False
    elif b1 is not None or b2 is not None:
        return False
    if n == 3:
        if m == 3:
            if s1[0] == s2[0] and s1[1] == s2[1] and s1[2] == s2[2]:
                result = True
            elif s1[0][0] == s2[0][0] and s1[1][0] == s2[1][0] and s1[2] == s2[2] and (len(s1[0]) == 1 or len(s2[0]) == 1):
                result = True
        elif m == 2:
            if s1[0] == s2[0] and s1[2] == s2[1]:
                result = True
            elif s1[0][0] == s2[0][0] and s1[2] == s2[1] and (len(s1[0]) == 1 or len(s2[0]) == 1):
                result = True
        else:
            if s1[2] == s2[0]:
                result = True
    elif n == 2:
        if m == 2:
            if s1[0] == s2[0] and s1[1] == s2[1]:
                result = True
            elif s1[0][0] == s2[0][0] and s1[1] == s2[1] and (len(s1[0]) == 1 or len(s2[0]) == 1):
                result = True
        else:
            if s1[1] == s2[0]:
                result = True
    elif n == 1:
        if s1[0] == s2[0]:
            result = True
    elif dist_jaccard(str1, str2) >= 0.9:
        result = True
    else:
        result = False
    
    return result

# Функция создания set`а групп именованных сущностей в тексте 
# (группа - одинаковый named entity, но по-разному написанный)
def make_groups(names):
    
    n = len(names)
    groups = np.array([-1] * n)
    gr = 1
    
    for i in range(n):
        
        j = i + 1
        
        while j != n:
            
            if compare_str(names[i], names[j]):
                if groups[i] == -1 and groups[j] == -1:
                    groups[i] = gr
                    groups[j] = gr
                    gr += 1
                elif groups[i] == -1:
                    groups[i] = groups[j]
                elif groups[j] == -1:
                    groups[j] = groups[i]
                elif groups[i] != groups[j]:
                    groups[np.where(groups == groups[j])] = groups[i]
                    
            j += 1
            
    for i in range(n):
        
        if groups[i] == -1:
            groups[i] = gr
            gr += 1
    
    set_of_groups = set(groups)
    result = []
    for group in set_of_groups:
        res_group = []
        indeces = np.where(groups == group)[0]
        for i in indeces:
            res_group.append(names[i])
        result.append(res_group)
    
    return result

# Функция выделения именованных сущностей из текста и распределения их по группам
def extract_names(text):
    
    extractor = NamesExtractor()
    matches = extractor(text)
    result = []
    pattern = re.compile('[\"\«\“](.+)[\"\»\”]')
    
    for match in matches:
        
        name = []
        
        if match.fact.first != None:
                name.append(match.fact.first)
        if match.fact.middle != None:
                name.append(match.fact.middle)
        if match.fact.last != None:
                name.append(match.fact.last)
                
        name = ' '.join(name).lower()
        result.append(name)
        
        start, stop = match.span
        text_to_find_book = text[max(start-75, 0):min(stop+75, len(text))]
        res_regexp = pattern.search(text_to_find_book)
        if res_regexp is not None:
            book = name + ' \"' + res_regexp.group(1) + '\"'
            result.append(book)
        
    return make_groups(result)

# Функция сравнения двух групп именовааных сущностей
def compare_groups(group1, group2):
    
    for el1 in group1:
        for el2 in group2:
            if compare_str(el1, el2):
                return True
    return False


# Функция создания матрицы: строки - тексты, столбцы - именованные сущности (а точнее группы)
def make_named_enteties_matrix(df):
    
    groups_in_matrix = np.array([])
    text_group_pairs = []
    n = len(df)
    
    
    for i in range(n):
        
        groups_in_text = extract_names(df.loc[i]['text'])
        
        for group_in_text in groups_in_text:
            
            is_group_found = False
            m = len(groups_in_matrix)
            
            for j in range(m):
                
                if compare_groups(group_in_text, groups_in_matrix[j]):
                
                    groups_in_matrix[j] = groups_in_matrix[j].union(set(group_in_text))
                    text_group_pairs.append((i,j))
                    is_group_found = True
                    break
                    
            if not is_group_found:
                groups_in_matrix = np.append(groups_in_matrix, set(group_in_text))
                text_group_pairs.append((i,m))
        
    m = len(groups_in_matrix)    
    matrix = np.zeros((n,m))
    
    for el in text_group_pairs:
        matrix[el] = 1
    
    columns = []
    for i in range(m):
        if np.sum(matrix[:,i]) > 1:
            columns.append(i)
    
    return matrix[:,columns], groups_in_matrix[columns]

In [25]:
%%time
named_entity_matrix, groups_in_matrix = make_named_enteties_matrix(df)
print(groups_in_matrix[:10])

[{'швабрин', 'алексей швабрин', 'алексей иванович швабрин'}
 {'гринев', 'пётр гринев'} {'онегин', 'евгений онегин'}
 {'пушкин "Евгений Онегин"', 'александр сергеевич пушкин "Евгений Онегин"', 'татьяна "Евгений Онегин"', 'евгений "Евгений Онегин"', 'евгений онегин "Евгений Онегин"'}
 {'евгений'}
 {'алексей швабрин "Капитанская дочка"', 'пётр гринев "Капитанская дочка"', 'швабрин "Капитанская дочка"', 'маша миронова "Капитанская дочка"', 'емельян пугачев "Капитанская дочка"', 'алексей иванович швабрин "Капитанская дочка"', 'пётр гринёв "Капитанская дочка"'}
 {'пётр'} {'пугачев', 'емельян пугачев'} {'ленский', 'владимир ленский'}
 {'данко'}]
Wall time: 1min 13s


### Составим матрицу Bag of Words для тем сочинений

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
vectorizer = CountVectorizer(binary = True, min_df = 0.01)

In [28]:
direction_matrix =  vectorizer.fit_transform(df['direction']).toarray()
print(direction_matrix.shape)

(179, 216)


In [29]:
words_in_directions = vectorizer.get_feature_names()

### Составим корреляционную матрицу между *topic_matrix* и *named_entity_matrix* 

In [30]:
from sklearn.metrics import matthews_corrcoef

In [31]:
m = named_entity_matrix.shape[1]
k = direction_matrix.shape[1]
corr_matrix = np.zeros((m,k))
print(named_entity_matrix.shape)
print(direction_matrix.shape)
print(corr_matrix.shape)

(179, 210)
(179, 216)
(210, 216)


In [32]:
for i in range(m):
    for j in range(k):
        corr_matrix[i,j] = matthews_corrcoef(named_entity_matrix[:,i], direction_matrix[:,j])

In [33]:
print(corr_matrix)

[[-0.0405269  -0.02849448 -0.0405269  ... -0.04544046 -0.02849448
   0.13901774]
 [-0.03049971 -0.02144436 -0.03049971 ... -0.03419756 -0.02144436
  -0.02633838]
 [-0.05362007 -0.0377003  -0.05362007 ... -0.06012109  0.13100854
  -0.04630424]
 ...
 [-0.01607087 -0.01129944 -0.01607087 ... -0.01801933 -0.01129944
   0.40015432]
 [-0.01607087 -0.01129944 -0.01607087 ... -0.01801933 -0.01129944
   0.40015432]
 [-0.01607087 -0.01129944 -0.01607087 ... -0.01801933 -0.01129944
  -0.01387818]]


In [34]:
z = 0
for i in range(m):
    for j in range(k):
        if corr_matrix[i,j] >= 0.7:
            print(groups_in_matrix[i], words_in_directions[j])
            z += 1
    if z >= 10:
        break

{'матрена "Матренин двор"', 'а и солженицын "Матренин двор"', 'александр иванович солженицын "Матренин двор"'} отзывчивость
{'матрена'} отзывчивость
{'сонечка мармеладова'} отзывчивость
{'джек лондон "Мартин Иден"', 'мартин "Мартин Иден"', 'мартин иден "Мартин Иден"'} голова
{'джек лондон "Мартин Иден"', 'мартин "Мартин Иден"', 'мартин иден "Мартин Иден"'} дар
{'мартин'} голова
{'мартин'} дар
{'мартин иден'} голова
{'мартин иден'} дар
{'илья ильич'} жильё


In [35]:
with open('Pickles\\named_entity_matrix.pickle', 'wb') as output:
    pickle.dump(named_entity_matrix, output)
    
with open('Pickles\\groups_in_matrix.pickle', 'wb') as output:
    pickle.dump(groups_in_matrix, output)

with open('Pickles\\vectorizer.pickle', 'wb') as output:
    pickle.dump(vectorizer, output)
    
with open('Pickles\\corr_matrix.pickle', 'wb') as output:
    pickle.dump(corr_matrix, output)

with open('Pickles\\direction_matrix.pickle', 'wb') as output:  
    pickle.dump(direction_matrix, output)